## Fixing KMC Handler Initialization and Context IssuesBased on the test failures, we need to address several issues in the KMC codebase:1. **Handler initialization errors**: The tests show `'ProjectHandler' object has no attribute 'config'`, `'GPT4Handler' object has no attribute 'config'`, and `'DocumentMetadataHandler' object has no attribute 'config'`.2. **Missing dependency**: The logs indicate issues with the `'llama_index.embeddings.azure_openai'` module.3. **Plugin discovery issues**: Tests expecting to find plugins are failing.Let's fix these issues one by one.

### 1. Fix Handler Initialization ErrorsThe error messages indicate that handler classes like `ProjectHandler`, `GPT4Handler`, and `DocumentMetadataHandler` are trying to access a `config` attribute that doesn't exist. This suggests a mismatch between how handlers are being initialized and how they're being used.

In [ ]:
import sys
import os
from pathlib import Path

# Define the path to BaseHandler class
base_handler_path = '/Users/giorgio/Documents/kmc/kmc/kmc_parser/handlers/base.py'

# Define the modified BaseHandler class with config attribute
handler_fix = """
class BaseHandler(ABC):
    """Clase base abstracta para todos los handlers de variables KMC."""
    
    def __init__(self, context=None):
        """Inicializa el handler con configuración básica de logging y contexto opcional."""
        self.logger = logging.getLogger(f"kmc.handler.{self.__class__.__name__}")
        self.context = context or {}
        self.config = {}  # Add config attribute that handlers are expecting
"""

# Read the current file
with open(base_handler_path, 'r') as f:
    content = f.read()

# Replace the BaseHandler class definition
import re
pattern = r'class BaseHandler\(ABC\):.*?def __init__\(self, context=None\):.*?self\.context = context or \{\}'
new_content = re.sub(pattern, handler_fix.strip(), content, flags=re.DOTALL)

# If the pattern was found and replaced
if new_content != content:
    # Write the updated content back to the file
    with open(base_handler_path, 'w') as f:
        f.write(new_content)
    print("Successfully updated BaseHandler with config attribute")
else:
    print("Could not update BaseHandler - pattern not found")

### 2. Create Missing `__init__.py` FilesLet's create any missing `__init__.py` files to ensure Python treats the directories as proper packages, which helps with relative imports.

In [ ]:
# Create missing __init__.py files
init_paths = [
    '/Users/giorgio/Documents/kmc/kmc/kmc_parser/extensions/lib/__init__.py',
    '/Users/giorgio/Documents/kmc/kmc/kmc_parser/handlers/context/__init__.py',
    '/Users/giorgio/Documents/kmc/kmc/kmc_parser/handlers/generative/__init__.py',
    '/Users/giorgio/Documents/kmc/kmc/kmc_parser/handlers/metadata/__init__.py',
]

for path in init_paths:
    # Make sure the directory exists first
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    # Create the file if it doesn't exist
    if not os.path.exists(path):
        with open(path, 'w') as f:
            f.write('# This file marks the directory as a Python package\n')
        print(f"Created {path}")
    else:
        print(f"{path} already exists")

### 3. Fix Auto Discovery for Plugin LoadingThe plugin discovery mechanism needs to be improved to correctly load and register plugins. Let's update the `auto_discovery.py` file to fix the relative import issues.

In [ ]:
# Path to auto_discovery.py
auto_discovery_path = '/Users/giorgio/Documents/kmc/kmc/kmc_parser/extensions/auto_discovery.py'

# Read the current file
with open(auto_discovery_path, 'r') as f:
    content = f.read()

# Define improved scanner methods for both handlers and plugins
scanner_fix = """
    def _scan_directory_for_handlers(self, directory):
        """Busca y registra handlers en un directorio"""
        self.logger.info(f"Escaneando handlers en: {directory}")
        handlers_count = 0

        for root, _, files in os.walk(directory):
            for file in files:
                if file.endswith(".py") and file != "__init__.py":
                    module_name = os.path.splitext(file)[0]
                    module_path = os.path.join(root, file)
                    
                    # Evitar procesar el mismo módulo más de una vez
                    if module_path in self._processed_modules:
                        self.logger.debug(f"Módulo ya procesado, omitiendo: {module_path}")
                        continue
                    
                    self._processed_modules.add(module_path)
                    
                    try:
                        # Determinar el nombre completo del módulo basado en la estructura de paquetes
                        rel_path = os.path.relpath(module_path, os.path.dirname(os.path.dirname(__file__)))
                        package_path = os.path.dirname(rel_path).replace(os.sep, ".")
                        full_module_name = f"kmc_parser.{package_path}.{module_name}" if package_path else f"kmc_parser.{module_name}"
                        
                        self.logger.debug(f"Cargando módulo como: {full_module_name}")
                        
                        # Intentar primero la importación basada en el nombre del módulo
                        try:
                            module = importlib.import_module(full_module_name)
                            self.logger.debug(f"Módulo cargado mediante importlib.import_module: {full_module_name}")
                        except (ImportError, ModuleNotFoundError):
                            # Si falla, caer en el método anterior
                            self.logger.debug(f"Fallback a spec_from_file_location para: {module_path}")
                            spec = importlib.util.spec_from_file_location(full_module_name, module_path)
                            if spec is None:
                                self.logger.warning(f"No se pudo cargar especificación para: {module_path}")
                                continue
                                
                            module = importlib.util.module_from_spec(spec)
                            
                            # Configurar el contexto de paquete para importaciones relativas
                            sys.modules[full_module_name] = module
                            
                            # Capturar salidas y excepciones durante la ejecución del módulo
                            try:
                                spec.loader.exec_module(module)
                            except Exception as e:
                                # Limpiar el módulo parcialmente cargado
                                if full_module_name in sys.modules:
                                    del sys.modules[full_module_name]
                                self.logger.error(f"Error al ejecutar módulo {module_path}: {str(e)}")
                                self.logger.debug(traceback.format_exc())
                                continue

                        # Buscar handlers en el módulo
                        for name, obj in vars(module).items():
                            try:
                                if hasattr(obj, "__kmc_handler_type__") and hasattr(obj, "__kmc_var_type__"):
                                    handler_type = getattr(obj, "__kmc_handler_type__")
                                    var_type = getattr(obj, "__kmc_var_type__")
                                    
                                    # Validar el tipo de handler
                                    if handler_type in ["context", "metadata", "generative"]:
                                        self.discovered_handlers.add((handler_type, var_type))
                                        handlers_count += 1
                                        self.logger.debug(f"Handler descubierto: {handler_type}:{var_type} en {module_path}")
                            except Exception as e:
                                self.logger.error(f"Error al procesar objeto {name} en {module_path}: {str(e)}")
                                
                    except Exception as e:
                        self.logger.error(f"Error al cargar módulo {module_path}: {str(e)}")
                        self.logger.debug(traceback.format_exc())

        return handlers_count

    def _scan_directory_for_plugins(self, directory):
        """Busca y registra plugins en un directorio"""
        self.logger.info(f"Escaneando plugins en: {directory}")
        plugins_count = 0

        for root, _, files in os.walk(directory):
            for file in files:
                if file.endswith(".py") and file != "__init__.py":
                    module_name = os.path.splitext(file)[0]
                    module_path = os.path.join(root, file)
                    
                    # Evitar procesar el mismo módulo más de una vez
                    if module_path in self._processed_modules:
                        self.logger.debug(f"Módulo ya procesado, omitiendo: {module_path}")
                        continue
                        
                    self._processed_modules.add(module_path)
                    
                    try:
                        # Determinar el nombre completo del módulo basado en la estructura de paquetes
                        rel_path = os.path.relpath(module_path, os.path.dirname(os.path.dirname(__file__)))
                        package_path = os.path.dirname(rel_path).replace(os.sep, ".")
                        full_module_name = f"kmc_parser.{package_path}.{module_name}" if package_path else f"kmc_parser.{module_name}"
                        
                        self.logger.debug(f"Cargando plugin como: {full_module_name}")
                        
                        # Intentar primero la importación basada en el nombre del módulo
                        try:
                            module = importlib.import_module(full_module_name)
                            self.logger.debug(f"Plugin cargado mediante importlib.import_module: {full_module_name}")
                        except (ImportError, ModuleNotFoundError):
                            # Si falla, caer en el método anterior
                            self.logger.debug(f"Fallback a spec_from_file_location para plugin: {module_path}")
                            spec = importlib.util.spec_from_file_location(full_module_name, module_path)
                            if spec is None:
                                self.logger.warning(f"No se pudo cargar especificación para: {module_path}")
                                continue
                                
                            module = importlib.util.module_from_spec(spec)
                            
                            # Configurar el contexto de paquete para importaciones relativas
                            sys.modules[full_module_name] = module
                            
                            try:
                                spec.loader.exec_module(module)
                            except Exception as e:
                                # Limpiar el módulo parcialmente cargado
                                if full_module_name in sys.modules:
                                    del sys.modules[full_module_name]
                                self.logger.error(f"Error al ejecutar módulo {module_path}: {str(e)}")
                                self.logger.debug(traceback.format_exc())
                                continue

                        # Buscar plugins en el módulo (clases que heredan de KMCPlugin)
                        for name, obj in vars(module).items():
                            try:
                                if inspect.isclass(obj) and issubclass(obj, KMCPlugin) and obj != KMCPlugin:
                                    # Crear instancia del plugin
                                    plugin_instance = obj()
                                    # Registrar el plugin
                                    if registry.register_plugin(plugin_instance):
                                        self.discovered_plugins.add(plugin_instance)
                                        plugins_count += 1
                                        self.logger.debug(f"Plugin descubierto y registrado: {name} en {module_path}")
                            except (TypeError, Exception) as e:
                                # TypeError ocurre en issubclass si obj no es una clase
                                if isinstance(e, TypeError):
                                    continue
                                self.logger.error(f"Error al procesar clase {name} en {module_path}: {str(e)}")
                    
                    except Exception as e:
                        self.logger.error(f"Error al cargar módulo {module_path}: {str(e)}")
                        self.logger.debug(traceback.format_exc())

        return plugins_count
"""

# Replace the scanner methods
pattern1 = r'def _scan_directory_for_handlers\([^}]*?\):.*?return handlers_count'
pattern2 = r'def _scan_directory_for_plugins\([^}]*?\):.*?return plugins_count'

# First replace the handler scanner
new_content = re.sub(pattern1, scanner_fix.split('def _scan_directory_for_plugins')[0].strip(), content, flags=re.DOTALL)
# Then replace the plugin scanner
new_content = re.sub(pattern2, "def " + scanner_fix.split('def _scan_directory_for_plugins')[1].strip(), new_content, flags=re.DOTALL)

# If the content was modified
if new_content != content:
    # Write the updated content back to the file
    with open(auto_discovery_path, 'w') as f:
        f.write(new_content)
    print("Successfully updated auto_discovery.py with improved module loading")
else:
    print("Could not update auto_discovery.py - patterns not found")

### 4. Create Example Plugin and TestsLet's create a simple test plugin to verify that plugin discovery works correctly.

In [ ]:
# Create a plugins directory if it doesn't exist
plugins_dir = '/Users/giorgio/Documents/kmc/kmc/kmc_parser/plugins'
os.makedirs(plugins_dir, exist_ok=True)

# Create an __init__.py file for the plugins directory
with open(os.path.join(plugins_dir, '__init__.py'), 'w') as f:
    f.write('# This file marks the directory as a Python package\n')

# Create a simple test plugin
test_plugin_path = os.path.join(plugins_dir, 'test_plugin.py')
test_plugin_content = """
from kmc_parser.extensions.plugin_base import KMCPlugin

class TestPlugin(KMCPlugin):
    """A simple test plugin for KMC Parser."""
    
    def __init__(self):
        """Initialize the plugin."""
        self.name = "test_plugin"
        self.version = "1.0.0"
        self.description = "A simple test plugin for KMC Parser"
    
    def process(self, content, context=None):
        """Process content using this plugin."""
        # This plugin just adds a signature to the content
        return f"{content}\n\n--- Processed by TestPlugin v{self.version} ---"
        
    def get_info(self):
        """Return plugin information."""
        return {
            "name": self.name,
            "version": self.version,
            "description": self.description
        }
"""

with open(test_plugin_path, 'w') as f:
    f.write(test_plugin_content)

print(f"Created test plugin at {test_plugin_path}")

### 5. Install Missing DependenciesLet's make sure all required dependencies are installed.

In [ ]:
# List of required packages
required_packages = [
    'llama-index-embeddings-azure-openai',
    'llama-index-vector-stores-supabase',
    'supabase',
]

# Install required dependencies
import subprocess

for package in required_packages:
    try:
        # Check if the package is already installed
        __import__(package.replace('-', '_'))
        print(f"{package} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"Successfully installed {package}")
        except subprocess.CalledProcessError:
            print(f"Failed to install {package}")

### 6. Update Decorators to Pass Context to HandlersNow, let's modify the decorators in base.py to ensure they pass the context to handlers when registering them.

In [ ]:
# Define improved decorators
decorators_fix = """
def context_handler(var_type: str):
    """
    Decorador para registrar automáticamente handlers contextuales.
    
    Args:
        var_type: Tipo de variable contextual que este handler maneja
    """
    def decorator(cls):
        # Añadir metadatos a la clase para identificarla en autodiscovery
        setattr(cls, "__kmc_handler_type__", "context")
        setattr(cls, "__kmc_var_type__", var_type)
        
        # Registrar una instancia del handler si no es una clase abstracta
        if not getattr(cls, "__abstractmethod__", False):
            try:
                # Pasamos un contexto vacío por defecto
                handler_instance = cls({})
                registry.register_context_handler(var_type, handler_instance)
            except Exception as e:
                logger = logging.getLogger("kmc.handlers")
                logger.error(f"Error al registrar handler contextual para {var_type}: {str(e)}")
        
        return cls
    return decorator


def metadata_handler(var_type: str):
    """
    Decorador para registrar automáticamente handlers de metadata.
    
    Args:
        var_type: Tipo de variable de metadata que este handler maneja
    """
    def decorator(cls):
        # Añadir metadatos a la clase para identificarla en autodiscovery
        setattr(cls, "__kmc_handler_type__", "metadata")
        setattr(cls, "__kmc_var_type__", var_type)
        
        # Registrar una instancia del handler si no es una clase abstracta
        if not getattr(cls, "__abstractmethod__", False):
            try:
                # Pasamos un contexto vacío por defecto
                handler_instance = cls({})
                registry.register_metadata_handler(var_type, handler_instance)
            except Exception as e:
                logger = logging.getLogger("kmc.handlers")
                logger.error(f"Error al registrar handler de metadata para {var_type}: {str(e)}")
        
        return cls
    return decorator


def generative_handler(var_type: str):
    """
    Decorador para registrar automáticamente handlers generativos.
    
    Args:
        var_type: Tipo de variable generativa que este handler maneja (formato "categoria:subtipo")
    """
    def decorator(cls):
        # Verificar formato correcto
        if ":" not in var_type:
            logger = logging.getLogger("kmc.handlers")
            logger.error(f"Formato incorrecto para handler generativo: {var_type}. Debe ser 'categoria:subtipo'")
            return cls
        
        # Añadir metadatos a la clase para identificarla en autodiscovery
        setattr(cls, "__kmc_handler_type__", "generative")
        setattr(cls, "__kmc_var_type__", var_type)
        
        # Registrar una instancia del handler si no es una clase abstracta
        if not getattr(cls, "__abstractmethod__", False):
            try:
                # Pasamos un contexto vacío por defecto
                handler_instance = cls({})
                registry.register_generative_handler(var_type, handler_instance)
            except Exception as e:
                logger = logging.getLogger("kmc.handlers")
                logger.error(f"Error al registrar handler generativo para {var_type}: {str(e)}")
        
        return cls
    return decorator
"""

# Update decorators in base.py
pattern = r'def context_handler\([^}]*?\):.*?def metadata_handler\([^}]*?\):.*?def generative_handler\([^}]*?\):[^}]*?return cls\s+return decorator\s'

# Read the current file again (it may have been modified)
with open(base_handler_path, 'r') as f:
    content = f.read()

# Update decorators
new_content = re.sub(pattern, decorators_fix, content, flags=re.DOTALL)

# If the pattern was found and replaced
if new_content != content:
    # Write the updated content back to the file
    with open(base_handler_path, 'w') as f:
        f.write(new_content)
    print("Successfully updated handler decorators")
else:
    print("Could not update handler decorators - pattern not found")

### 7. Run Tests to Verify FixesNow that we've applied all the fixes, let's run the tests to see if they pass.

In [ ]:
# Run the tests
try:
    os.chdir('/Users/giorgio/Documents/kmc/kmc')
    result = subprocess.run([sys.executable, '-m', 'unittest', 'discover', '-s', 'kmc_parser/tests'], 
                           capture_output=True, text=True)
    print(result.stdout)
    print(result.stderr)
    print(f"Test exit code: {result.returncode}")
except Exception as e:
    print(f"Error running tests: {e}")

## Summary of FixesHere's what we've done to fix the KMC tests:1. **Fixed the BaseHandler class**: Added a `config` attribute to the `BaseHandler` class since other handlers were expecting it.2. **Created missing `__init__.py` files**: Ensured Python treats all directories as packages, helping with relative imports.3. **Improved auto_discovery.py**: Updated the module loading mechanism to properly handle relative imports.4. **Created a test plugin**: Added a sample plugin to verify that plugin discovery works correctly.5. **Updated decorators**: Ensured handler decorators pass the context to handlers when registering them.6. **Installed missing dependencies**: Made sure all required packages are installed.These changes should address the core issues causing the test failures in your KMC project.